<a href="https://colab.research.google.com/github/ogut77/LogisticRegression/blob/master/LightGBM_Undersampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
# Grafik çizdirmek için gerekli paket.
import matplotlib.pyplot as plt
# LightGBM paketi.
import lightgbm as lgb
import numpy as np
#Uyarı mesajlarını kapatmak için
import warnings
warnings.filterwarnings("ignore")
# Veri çerçevelerini daha güzel görselleştirmek için
from IPython.display import display, HTML
# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
d2 = pd.read_csv('/content/drive/My Drive/Santander/train.csv')
d2.drop(['Unnamed: 0'], axis = 1, inplace = True)
d2.drop(['ID_code'], axis = 1, inplace = True)
d2.head(2)

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,0,8.9255,-6.7863,11.9081,5.093,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,0,11.5006,-4.1473,13.8588,5.389,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518


In [4]:
from sklearn.model_selection import train_test_split

df, te = train_test_split(d2, test_size=0.2)
df.shape

(160000, 201)

In [0]:
no_frauds = len(df[df['target'] == 1])
non_fraud_indices = df[df.target == 0].index
import numpy as np
random_indices = np.random.choice(non_fraud_indices,no_frauds, replace=False)
fraud_indices = df[df.target == 1].index
under_sample_indices = np.concatenate([fraud_indices,random_indices])
under_sample = df.loc[under_sample_indices]
df=under_sample.copy()


In [0]:
X_train=df.iloc[:,1:]
X_test=te.iloc[:,1:]
y_train=df.iloc[:,0:1]
y_test=te.iloc[:,0:1]

In [0]:
print(d2.isnull().sum())

In [0]:
lgb_train = lgb.Dataset(data=X_train, label=y_train,  free_raw_data=False)

In [0]:
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'rf',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}


In [0]:
gbm = lgb.train(parameters ,lgb_train)

In [0]:
y_pred = gbm.predict(X_test)

print('Eğri altı alan değeri:', roc_auc_score(y_test, y_pred))
print('İsabetlilik değeri:', accuracy_score(y_test, ( y_pred>= 0.5)*1))

Eğri altı alan değeri: 0.7726133550297062
İsabetlilik değeri: 0.123425


In [0]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'verbose': -1
}

In [0]:
gbm = lgb.train(params ,lgb_train)

In [10]:
y_pred = gbm.predict(X_test)

print('Eğri altı alan değeri:', roc_auc_score(y_test, y_pred))
print('İsabetlilik değeri:', accuracy_score(y_test, ( y_pred>= 0.5)*1))

Eğri altı alan değeri: 0.8434023493033269
İsabetlilik değeri: 0.747325


In [13]:
# Değerlendirme veri kümesini oluşturuyoruz.
lgb_eval = lgb.Dataset(data=X_test, label=y_test, reference=lgb_train,  free_raw_data=False)

# Eğitim parametrelerini belirleyelim
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'verbose': -1
}

print('Eğitim...')
# Modeli eğitelim
# Bu sefer değerlendirme veri kümesini de tanıtıyoruz.
evals_result={}
gbm = lgb.train(params,
                lgb_train,
                valid_sets = lgb_eval,
                num_boost_round= 5000,
                early_stopping_rounds= 100,
                evals_result=evals_result)
print('Eğitim bitti...')

# Tahmin ve değerlendirme
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

print('En iyi sonucu veren iterasyon: ', gbm.best_iteration)
print('Eğri altı alan değeri:', roc_auc_score(y_test, y_pred))
print('İsabetlilik değeri:', accuracy_score(y_test, ( y_pred>= 0.5)*1))

Eğitim...
[1]	valid_0's auc: 0.65633
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.681163
[3]	valid_0's auc: 0.682112
[4]	valid_0's auc: 0.688665
[5]	valid_0's auc: 0.695622
[6]	valid_0's auc: 0.70185
[7]	valid_0's auc: 0.708581
[8]	valid_0's auc: 0.71421
[9]	valid_0's auc: 0.7189
[10]	valid_0's auc: 0.722901
[11]	valid_0's auc: 0.728278
[12]	valid_0's auc: 0.73335
[13]	valid_0's auc: 0.736215
[14]	valid_0's auc: 0.740867
[15]	valid_0's auc: 0.744085
[16]	valid_0's auc: 0.747929
[17]	valid_0's auc: 0.751919
[18]	valid_0's auc: 0.755758
[19]	valid_0's auc: 0.7595
[20]	valid_0's auc: 0.762312
[21]	valid_0's auc: 0.765892
[22]	valid_0's auc: 0.766939
[23]	valid_0's auc: 0.769523
[24]	valid_0's auc: 0.772078
[25]	valid_0's auc: 0.774662
[26]	valid_0's auc: 0.777368
[27]	valid_0's auc: 0.779405
[28]	valid_0's auc: 0.780703
[29]	valid_0's auc: 0.78231
[30]	valid_0's auc: 0.785075
[31]	valid_0's auc: 0.787331
[32]	valid_0's auc: 0.788986
[33]	valid_0's au